# Анализ уровня потребительской лояльности (NPS).

**Ссылка на проект в Tableau**
<https://public.tableau.com/app/profile/ekaterina.nosova5462/viz/NPS_17281217676470/NPS_3>

## Описание проектной работы.

Заказчик исследования — большая телекоммуникационная компания, которая оказывает услуги на территории всего СНГ. Стоит задача определить текущий уровень потребительской лояльности, или NPS (от англ. Net Promoter Score), среди клиентов из России. 

Чтобы определить уровень лояльности, клиентам задавали классический вопрос: «Оцените по шкале от 1 до 10 вероятность того, что вы порекомендуете компанию друзьям и знакомым».

Чтобы оценить результаты опроса, оценки обычно делят на три группы:
- 9-10 баллов — «cторонники» (англ. promoters);
- 7-8 баллов — «нейтралы» (англ. passives);
- 0-6 баллов — «критики» (англ. detractors).

Итоговое значение NPS рассчитывается по формуле: *% «сторонников» - % «критиков»*.

Таким образом, значение этого показателя варьируется от -100% (когда все клиенты «критики») до 100% (когда все клиенты лояльны к сервису). Но это крайние случаи, которые редко встретишь на практике. 

## Описание данных

- 'user_id' Идентификатор клиента
- 'lt_day' Количество дней «жизни» клиента
- 'is_new' Поле хранит информацию о том, является ли клиент новым/ старым (новый клиент -если количество дней «жизни» составляет не более 365 дней)
- 'age'	Возраст
- 'gender_segment'	Пол (мужской / женский)
- 'os_name'	Тип операционной системы
- 'cpe_type_name'	Тип устройства
- 'country'	Страна проживания
- 'city'	Город проживания
- 'age_segment'	Возрастной сегмент
- 'traffic_segment'	Сегмент по объёму потребляемого трафика
- 'lifetime_segment' Сегмент по количеству дней «жизни» (в месяцах)
- 'nps_score' Оценка клиента в NPS-опросе
- 'nps_group' Поле хранит информацию о том, к какой группе относится оценка клиента в опросе (критик/ нейтрал/ сторонник).

## Состав и описание выполнения работы.

### Шаг 1. Подключение к базе

In [1]:
#Импорт библиотек
import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [2]:
path_to_db_local = 'telecomm_csi.db'
path_to_db_platform = '/datasets/telecomm_csi.db'
path_to_db = None

if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
else:
    raise Exception('Файл с базой данных SQLite не найден!')

if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

### Шаг 2. Выгрузка данных.

In [3]:
query = """
SELECT 
    u.user_id,
    u.lt_day,
    CASE 
        WHEN u.lt_day <= 365 THEN 'Новый' 
        ELSE 'Старый' 
    END AS is_new,
    u.age,
    CASE 
        WHEN u.gender_segment = 1 THEN 'женщина'
        ELSE 'мужчина'
    END AS gender_segment,
    u.os_name,
    u.cpe_type_name,
    l.country,
    l.city,
    ag.title AS age_segment,
    tr.title AS traffic_segment,
    lt.title AS lifetime_segment,
    u.nps_score,
    CASE 
        WHEN u.nps_score >= 9 THEN 'Сторонник'
        WHEN u.nps_score >= 7 THEN 'Нейтрал'
        ELSE 'Критик'
    END AS nps_group
FROM 
    user u
JOIN 
    location l ON u.location_id = l.location_id
JOIN 
    age_segment ag ON u.age_gr_id = ag.age_gr_id
JOIN 
    traffic_segment tr ON u.tr_gr_id = tr.tr_gr_id
JOIN 
    lifetime_segment lt ON u.lt_gr_id = lt.lt_gr_id;
"""

In [4]:
df = pd.read_sql(query, engine)
df.head(5)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,Старый,45.0,женщина,ANDROID,SMARTPHONE,Россия,Уфа,05 45-54,04 1-5,08 36+,10,Сторонник
1,A001WF,2344,Старый,53.0,мужчина,ANDROID,SMARTPHONE,Россия,Киров,05 45-54,04 1-5,08 36+,10,Сторонник
2,A003Q7,467,Старый,57.0,мужчина,ANDROID,SMARTPHONE,Россия,Москва,06 55-64,08 20-25,06 13-24,10,Сторонник
3,A004TB,4190,Старый,44.0,женщина,IOS,SMARTPHONE,Россия,РостовнаДону,04 35-44,03 0.1-1,08 36+,10,Сторонник
4,A004XT,1163,Старый,24.0,мужчина,ANDROID,SMARTPHONE,Россия,Рязань,02 16-24,05 5-10,08 36+,10,Сторонник


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502493 entries, 0 to 502492
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   user_id           502493 non-null  object 
 1   lt_day            502493 non-null  int64  
 2   is_new            502493 non-null  object 
 3   age               501939 non-null  float64
 4   gender_segment    502493 non-null  object 
 5   os_name           502493 non-null  object 
 6   cpe_type_name     502493 non-null  object 
 7   country           502493 non-null  object 
 8   city              502493 non-null  object 
 9   age_segment       502493 non-null  object 
 10  traffic_segment   502493 non-null  object 
 11  lifetime_segment  502493 non-null  object 
 12  nps_score         502493 non-null  int64  
 13  nps_group         502493 non-null  object 
dtypes: float64(1), int64(2), object(11)
memory usage: 53.7+ MB


In [6]:
df.isna().sum().sort_values(ascending=False) #информация о количестве пропусков в датафрейме df

age                 554
user_id               0
lt_day                0
is_new                0
gender_segment        0
os_name               0
cpe_type_name         0
country               0
city                  0
age_segment           0
traffic_segment       0
lifetime_segment      0
nps_score             0
nps_group             0
dtype: int64

Количество пропущенных значений по полю 'age' менее 1% (0,1%), следовательно, исключение пропущенных значений не повлечет серьезного искажения информации.

На уровне источника данных отфильтрую таблицу, исключив по полю 'age_segment' сегмент '08 n/a'.

In [7]:
df['os_name'].unique().tolist()

['ANDROID',
 'IOS',
 'OTHER',
 'unknown',
 'PROPRIETARY',
 'WINDOWS PHONE',
 'SYMBIAN OS',
 'BADA OS',
 'WINDOWS MOBILE']

In [8]:
df['cpe_type_name'].unique().tolist()

['SMARTPHONE',
 'TABLET',
 'MOBILE PHONE/FEATURE PHONE',
 'PHONE',
 'ROUTER',
 'MODEM',
 'WLAN ROUTER',
 'USB MODEM',
 'unknown',
 'PORTABLE(INCLUDE PDA)',
 'NETWORK DEVICE',
 'MIFI ROUTER',
 'MOBILE TEST PLATFORM',
 'HANDHELD']

Поля 'os_name' и 'cpe_type_name' содержат пропущенные значения, которые заменены на 'unknown'.

In [9]:
df['city'].unique().tolist()

['Уфа',
 'Киров',
 'Москва',
 'РостовнаДону',
 'Рязань',
 'Омск',
 'СанктПетербург',
 'Волгоград',
 'Тольятти',
 'Казань',
 'Самара',
 'Красноярск',
 'Екатеринбург',
 'Калуга',
 'Краснодар',
 'Иркутск',
 'Пермь',
 'Владимир',
 'Ижевск',
 'Тюмень',
 'Оренбург',
 'НижнийНовгород',
 'Брянск',
 'Челябинск',
 'Астрахань',
 'Сургут',
 'Тверь',
 'Новосибирск',
 'НабережныеЧелны',
 'Махачкала',
 'Воронеж',
 'Курск',
 'Владивосток',
 'Балашиха',
 'Пенза',
 'Калининград',
 'Тула',
 'Саратов',
 'Кемерово',
 'Белгород',
 'Барнаул',
 'Чебоксары',
 'Архангельск',
 'Томск',
 'Ярославль',
 'Ульяновск',
 'Хабаровск',
 'Грозный',
 'Ставрополь',
 'Липецк',
 'Новокузнецк',
 'Якутск',
 'УланУдэ',
 'Сочи',
 'Иваново',
 'НижнийТагил',
 'Смоленск',
 'Волжский',
 'Магнитогорск',
 'Чита',
 'Череповец',
 'Саранск']

In [10]:
#Сохранение выгруженной таблицы
df.to_csv('telecomm_csi_tableau.csv', index=False)

### Шаг 3. Создание дашборда в Tableau.

#### Дашборд "Распределение участников опроса".

Отвечает на вопросы:
- Как распределены участники опроса по возрасту и полу?
- Каких пользователей больше: новых или старых?
- Пользователи из каких городов активнее участвовали в опросе?

#### Дашборд "Значение NPS".

Отвечает на вопросы:
- Какие группы пользователей наиболее лояльны к сервису? Какие менее?
- Какой общий NPS среди всех опрошенных?

#### Дашборд "Клиенты-сторонники сервиса".

Описывает клиентов, которые относятся к группе cторонников.